In [12]:
!python3 -m pip install clickhouse-driver --user

     |████████████████████████████████| 150 kB 2.5 MB/s eta 0:00:01


In [44]:
class MapWay:
    
    
    def __init__(self, 
                 nameAlgo=None, 
                 arrAsks=None, 
                 current_position=None, 
                 points=None, 
                 model=None, 
                 places=None):
        self.nameAlgo = nameAlgo
        self.arrAsks = arrAsks
        self.points = points
        self.model = model
        self.places = places
        self.infinity = 99999999999912
    
    def findBestWay(self, user=None, points=None):
        '''
        user: пользователь для которого мы анализируем
        points: пока что массив на две точки
        [0]: начальная
        [1]: конечная
        '''
        # поиск объектов, которые попадают под интересующую нас зону
        places_in_zone = []  
        for place in self.places:
            if inZone(first_point=points[0], 
                      second_point=points[1], 
                      object_place = place):
                places_in_zone.append(place)
    
    
        '''
            Добавление начальной и конечной точки, 
            можно будет добавить промежуточную
        '''
        for point in points:
            places_in_zone.append(point)
        '''
        Построение графа исходя из модели
        
        1) Возьмём и посмотрим насколько подходит пользователю каждое место из предложенных,
            для этого будем использовать ранг каждого места для посетителя, рачитывать его мы будем по формуле,
            { Формулу я ещё не придумал }
            
        '''
        self.matrix_size_x = len(places_in_zone)
        matrix_places = [[0] * matrix_size_x] * matrix_size_x
        
        for i in range(matrix_size_x):
            for j in range(matrix_size_x):
                '''
                    Добаляю расстояния в матрицу
                    
                    !!!!  Здесь нам надо будет брать время, 
                    которое понадобится для прохода из 
                    одной точки в другую с яндекс карт  !!!!
                    
                    Расстояние думаю лучше измерять во времени, которое человек затратит на путь
                '''
                if(((places_in_zone[i].category == 'Start_point' and places_in_zone[j] == 'End_point' ) or 
                   (places_in_zone[i].category == 'End_point' and places_in_zone[j] == 'Start_point' )) and 
                   self.matrix_size_x > 2):
                    matrix_places[i][j] = self.infinity
                else:
                    matrix_places[i][j] = getLength(places_in_zone[i], places_in_zone[j]) 
                    
        '''
        
        S
         \
          \
           \
            *-------E

        Буду использовать алгоритм дейкстры для поиска пути, но буду так же искать по точке, 
        которая больше всего нравится пользователю и по времени до конца
        '''
        
        used = [True] * self.matrix_size_x
        d = []
        for i in range(self.matrix_size_x):
            d.append(GraphType(length=self.infinity, 
                               vertex_number=i))
        s_point = self.matrix_size_x - 2
        end_point = self.matrix_size_x - 1
        d[s_point].length = 0
        for i in range(self.matrix_size_x):
            
            
            v = None
            for j in range(self.matrix_size_x):
                if not used[j] and (v == None or d[j].length < d[v].length):
                    v = j
            if d[v].length == self.infinity:
                break
            used[v] = True
            
            for e in range(len(matrix_places[v])):
                if d[v].length + matrix_places[v][e] + places_in_zone[e].visit_time < d[e].length:
                    d[e].length = d[v].length + matrix_places[v][e] + places_in_zone[e].visit_time
                    
            '''
                По тому, что выше можно построить путь из точек, 
                если добавить несколько элементов
            '''
            
        '''
        
        Чтобы восстановить оптимальный путь от 
        начальной вершины до коненой нам нужно просто смотреть на массив
        в котором у нас записаны минимальные расстояния до начальной вершины
        Мы будем жадно брать самое маленькое значение
    
        '''
        points_in_way = []

        sorted_d = sorted(d, key=lambda item: item.length)
        sorted_d.reverse()
        
        end_point_of_object = None
        for i in range(len(sorted_d)):
            if sorted_d[i].vertex_number == end_point:
                end_point_of_object = i
                break
        for i in range(end_point_of_object, len(sorted_d)):
            points_in_way.append(sorted_d[i].vertex_number)
            
        minimal_time_to_place = d[end_point]
        return (minimal_time_to_place, points_in_way)
    
    def inZone(self, 
               first_point=None,
               second_point=None, 
               object_place=None):
        '''
            TODO: добавить функцию просмотра в окружении на точках, для захвата большего количества возможных мест
        
            y                                         y
            ^                                         ^
            X----------------------                   --------------------Y
            |         *           |                   |   *               |   *
            |                     | *            *    |            *      |
            |    *         *      |                   |                   |
            |                     |                   |       *           |
        *   |                     |                   |                   |
            |         *       *   |    *              |                   | *
            |                     |               *   |   *               |
          * ----------------------Y > x               X-------------------- > x
          
        '''
        left_point = None
        right_point = None
        if first_point.longtitude <= second_point.longtitude:
            left_point = first_point
        else:
            left_point = second_point
        
        if left_point.latitude >= right_point.latitude:
            if (object_place.longtitude <= right_point.longtitude and 
                object_place.longtitude >= left_point.longtitude and 
                object_place.latitude <= left_point.latitude and 
                object_place.latitude >= right_point.latitude):
                return True
            else:
                return False
        else:
            if (object_place.longtitude <= right_point.longtitude and 
                object_place.longtitude >= left_point.longtitude and 
                object_place.latitude <= right_point.latitude and 
                object_place.latitude >= left_point.latitude):
                return True
            else:
                return False
    
    def getLength(self, first_point, second_point):
        import math
        return math.sqrt((first_point.latitude - second_point.latitude) ** 2 + 
                (first_point.longtitude - second_point.longtitude) ** 2)
    
    
    
    '''
    Ниже я буду пытаться сам написать алгоритм 
    поиска пути исходя из наших критериев поиска
    а именно предпочтений пользователя и времени
    '''
    def evg_algo(self):
        '''
        
            Надо взять как минимум одну достопримечательность + (Сделал бесконечное расстояние до конечной точки)
            если она имеется в предложенной области
            и смотреть на время, которое указал пользователь при вводе
            при этом надо взять самые интересные, поэтому мы должны 
            отсортировать объекты по степени их интересности и смотреть
            от самого интересного к менее интересному

            ----------------------------------
            |                                |
            |     S----\                     |
            |          -[]                   |
            |                                |
            |     []           []            |
            |                       []       |
            |                                |
            |            []                  |
            |                         E      |
            |                                |
            ----------------------------------
        '''
        
        pass
    
    '''
    Ниже представлены алгоритм Куна, думал его можно как-то 
    использовать для поиска удовлетворяющего пути, 
    но не думаю что он будет эффективен
    '''
    def kunAlgo(self):
        '''
        Здесь тоже ошибки, потому что алгоритм написан для общего
        случая, надо переделать под нашу ситуацию
        '''
        fill(matching, -1)
        for i in range(n):
            fill(used, False)
            dfs(i)
        for i in range(n):
            if(matching[i] != - 1):
                print(i + " " + matching[i])
        pass
    
    def fill(self, graph, number):
        '''
        Для заполнения матрицы каким-то одним значением
        '''
        for i in range(len(graph)):
            for j in range(len(graph[i])):
                graph[i][j] = number
    
    def dfs(self, v:int):
        '''
        Здесь ошибка
        '''
        if (used[v]):
            return False
        used[v] = True
        for to in g[v]:
            if (matching[to] == -1 or dfs(matching[to])): #ошбика где dfs(matching[to])
                matching[to] = v
                return True
        return False
    
class WorkWithDB:
    
    def __init__(self):
        pass
    
class User:
    
    def __init__(self, 
                 time=None, 
                 age=None, 
                 religion=None, 
                 sex=None, 
                 marital_status=None, 
                 other_info=None):
        '''
            Create or get dataset
            Сделал все метрики для лучшего понимания что собирать с людей
        '''
#         self.category = model.predict()
     
    def getCategory(self):
        return self.category

    
class GraphType:
    
    def __init__(self, 
                 length=None, 
                 vertex_number=None):
        self.length = length
        self.vertex_number = vertex_number
        
    
class Position:
    
    def __init__(self, 
                 latitude=None, 
                 longtitude=None, 
                 type_of_point=None):
        self.latitude = latitude
        self.longtitude = longtitude
        self.type_of_point = None

class Place:
    '''
        ---------------------------------------------
        |                                           |
        |   Категории мест:                         |
        |       Start_point: начальная точка        |
        |           пути                            |
        |       Middle_point: точки, которые        |
        |           пользователь отметил между      |
        |           начальной и конечной            |
        |       End_point: конечная точка пути      |
        |       Place_point: точка с местом         |
        |                                           |
        |                                           |
        ---------------------------------------------
    '''
    
    def __init__(self, 
                 name=None, 
                 position : Position=None, 
                 category=None, 
                 rank=None, 
                 length=None, 
                 visit_time=None):
        self.name = name
        self.position = position
        self.category = category
        self.length = length
        self.rank = rank
        seff.visit_time = visit_time

class Metrics_and_algos:
    
    def __init__(self):
        pass
    
    def confustion_matrix(self):
        pass
    
    def accuracy_metric(self):
        '''
        Эту метрику можно назвать базовой. 
        Она измеряет количество верно классифицированных объектов 
        относительно общего количества всех объектов.
        '''
        pass
    
    def sensitivity_metric(self):
        '''
        Сколько объектов наша модель смогла правильно классифицировать 
        с позитивной меткой из всего множества позитивных.
        '''
        pass
    
    def precision_metric(self):
        '''
        Сколько из всех объектов, которые классифицируются как положительные, 
        действительно являются положительными, относительно общего 
        количества полученных от модели позитивных меток.
        '''
        pass
    
    def f1_score(self):
        '''
        Сочетание precision и recall, 
        дает некоторый компромисс между ними двумя, 
        оценка F1 достигает своего наилучшего значения в 1 и худшее в 0.
        '''
        pass

In [49]:
a = [3,3,3,32,1,2]
b = [5,3,1]
for i in b:
    a.append(i)

def fill(graph, number):
    for i in range(len(graph)):
        for j in range(len(graph[i])):
            graph[i][j] = number
            
graph = [
    [2, 4 ,3],
    [5, 1, 7],
    [6, 3, 1]
]
fill(graph, -1)
graph

a = [3,3,3,32,1,2]
a.reverse()
a

for i in range(2, len(a)):
    print(a[i])

32
3
3
3
